# Understanding Custom Space

CustomSpace is the tool to incorporate vectors created outside the Superlinked system.

In [1]:
%pip install superlinked==37.4.1

In [2]:
from superlinked import framework as sl

## Creating a custom space

To use pre-created custom vectors in Superlinked:
* create a schema (just like in any other case) with a `FloatList` type `SchemaField`,
* create a `CustomSpace` based on that field. You only need to set the length of the `FloatList`, meaning the number of scalars your vectors (referring to 1 entity) consist of,
* working with Superlinked, vectors often need to be aggregated (see [event effects](event_effects.ipynb) for example) or normalized.
* During aggregation vectors are summed up elementwise, and normalized using L2 norm of the vector to achieve unit vector length when needed.

In [3]:
class Product(sl.Schema):
    id: sl.IdField
    sales_history: sl.FloatList  # we need to supply vectors using this SchemaFieldType


product = Product()

In [4]:
sales_history_space = sl.CustomSpace(
    vector=product.sales_history,
    length=3,
)
product_index = sl.Index(sales_history_space)

In [5]:
source: sl.InMemorySource = sl.InMemorySource(product)
executor = sl.InMemoryExecutor(sources=[source], indices=[product_index])
app = executor.run()

In [6]:
# FloatList can be parsed from list or np.array of floats or ints
source.put(
    [
        {"id": "product-1", "sales_history": [1.0, 0.0, 0.0]},
        {"id": "product-2", "sales_history": [0.0, 0.8, 0.2]},
        {"id": "product-3", "sales_history": [0.1, 0.1, 0.8]},
    ]
)

At query-time, we can search with any vector that has the correct length, this is demonstrated here by making all of the 3 products be the first result once.

In [7]:
sales_history_query = (
    sl.Query(product_index).find(product).similar(sales_history_space, sl.Param("sales_history")).select_all()
)

In [8]:
result = app.query(sales_history_query, sales_history=[1.0, 0.0, 0.0])
sl.PandasConverter.to_pandas(result)

sales_history         id  similarity_score
0  [1.0, 0.0, 0.0]  product-1          1.000000
1  [0.1, 0.1, 0.8]  product-3          0.123091
2  [0.0, 0.8, 0.2]  product-2          0.000000

In [9]:
result = app.query(sales_history_query, sales_history=[0.0, 1.0, 0.0])
sl.PandasConverter.to_pandas(result)

sales_history         id  similarity_score
0  [0.0, 0.8, 0.2]  product-2          0.970143
1  [0.1, 0.1, 0.8]  product-3          0.123091
2  [1.0, 0.0, 0.0]  product-1          0.000000

In [10]:
result = app.query(sales_history_query, sales_history=[0.0, 0.0, 1.0])
sl.PandasConverter.to_pandas(result)

sales_history         id  similarity_score
0  [0.1, 0.1, 0.8]  product-3          0.984732
1  [0.0, 0.8, 0.2]  product-2          0.242536
2  [1.0, 0.0, 0.0]  product-1          0.000000